In [99]:

from simulation.functions import *



base = 'USD'
portfolio_name = 'portfolio' + base

data = get_portfolio_data(portfolio_name,base)



factors = data[6]
factors.to_csv('factors.csv')

In [94]:
import pandas as pd


df = pd.read_csv('factors.csv')
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df.set_index('date', inplace=True)
df['dollar_shift'] = df['dollar'].shift(-1)
df['carry_shift'] = df['carry'].shift(-1)

df = df.dropna()

df = pd.concat([df,df_hedge],axis = 1,join = 'inner')
predictors = df[['volatility','afd','commodity','ted']]
predictors_factors = df[['dollar','carry']]

dollar = df[['dollar_shift']]
carry = df[['carry_shift']]

CHF = df[['CHF']]
SEK = df[['SEK']]
NOK = df[['NOK']]
NZD = df[['NZD']]
AUD = df[['AUD']]
EUR = df[['EUR']]
GBP = df[['GBP']]
JPY = df[['JPY']]
CAD = df[['CAD']]



In [96]:
import statsmodels.api as sm
import pandas as pd

# Rolling OLS über multiple Fenster und p-Wert-Erfassung
import statsmodels.api as sm
import pandas as pd

def rolling_ols_avg_pvalues(ziel, pred, window=60):
    pvalues_list = []
    r2_list = []

    for t in range(len(ziel) - window + 1):
        window_pred = pred.iloc[t:t+window].copy()
        window_ziel = ziel.iloc[t:t+window].copy()

        # Add constant (Intercept)
        X = sm.add_constant(window_pred)
        y = window_ziel

        # Fit model
        model = sm.OLS(y, X).fit()

        # Save p-values and R²
        pvalues_list.append(model.pvalues)
        r2_list.append(model.rsquared)

    # Combine all p-values into DataFrame
    pval_df = pd.DataFrame(pvalues_list, index=ziel.index[window-1:])
    mean_pvals = pval_df.mean()

    x = r2_list
    # carry
    R = sum(x)/ len(x)

    return mean_pvals, pval_df, R

# Anwendung auf dein DataFrame
# Beispiel: factorsUSD muss vorher existieren
mean_pvalues, all_pvalues, R = rolling_ols_avg_pvalues(dollar,predictors)

# Ausgabe
print("Durchschnittliche p-Werte über alle Fenster:")
print(mean_pvalues)

print('durchscnittlicher R^2:',R)

Durchschnittliche p-Werte über alle Fenster:
const         0.533299
volatility    0.525830
afd           0.372355
commodity     0.350143
ted           0.605860
dtype: float64
durchscnittlicher R^2: 0.08287976689134922


In [89]:
import shap
import xgboost as xgb
import pandas as pd
import numpy as np

def rolling_xgboost_shap(ziel, pred, window=60, params=None):
    if params is None:
        params = {"n_estimators": 100, "max_depth": 2, "learning_rate": 0.5}

    shap_list = []
    r2_list = []

    for t in range(len(ziel) - window + 1):
        # Rolling Window
        X_window = pred.iloc[t:t+window]
        y_window = ziel.iloc[t:t+window]

        # Modell trainieren
        model = xgb.XGBRegressor(**params)
        model.fit(X_window, y_window)

        # SHAP-Werte berechnen
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_window)

        # Mittelwert der absoluten SHAP-Werte pro Feature
        shap_mean = np.abs(shap_values).mean(axis=0)
        shap_list.append(shap_mean)

        # Optional: Modellgüte (R²)
        r2 = model.score(X_window, y_window)
        r2_list.append(r2)

    # DataFrame: SHAP-Werte im Zeitverlauf
    shap_df = pd.DataFrame(shap_list, columns=pred.columns, index=ziel.index[window-1:])
    shap_avg = shap_df.mean()

    # Durchschnittliches R²
    r2_avg = np.mean(r2_list)

    return shap_avg, shap_df, r2_avg

In [97]:
shap_avg, shap_over_time, r2_mean = rolling_xgboost_shap(dollar, predictors)

print("⏱ Durchschnittliche SHAP-Werte:")
print(shap_avg)

print(f"📈 Durchschnittliches R²: {r2_mean:.4f}")

⏱ Durchschnittliche SHAP-Werte:
volatility    0.005794
afd           0.004272
commodity     0.004749
ted           0.004347
dtype: float32
📈 Durchschnittliches R²: 0.9951
